In [1]:
import brightway2 as bw
import perdu as p

In [2]:
bw.projects.set_current("US EPA")

In [9]:
bw.bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/25/2019 11:22:22
  Finished: 07/25/2019 11:22:23
  Total time elapsed: 00:00:00
  CPU %: 90.60
  Memory %: 1.12
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.30 seconds
Wrote 850 LCIA methods with 220699 characterization factors
Creating core data migrations



US LCI import following [the example notebook](https://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/IO%20-%20Importing%20the%20US%20LCI%20database.ipynb)

In [22]:
us_lci = "/Users/cmutel/Documents/LCA/US LCI database/2014"

In [23]:
ei = bw.SingleOutputEcospold1Importer(us_lci, "US LCI")

Extracting XML data from 702 datasets
Extracted 702 datasets in 0.87 seconds


In [24]:
bad_files = (
    '/Users/cmutel/Documents/LCA/US LCI database/2014/Spring wheat straw, production, average, US, 2022.xml',
    '/Users/cmutel/Documents/LCA/US LCI database/2014/Harvesting, fresh fruit bunch, at farm.xml',
)
ei.data = [obj for obj in ei.data if obj.get('filename') not in bad_files]

In [25]:
ei.migrate("us-lci")

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


In [26]:
ei.apply_strategies()

Applying strategy: normalize_units
Applying strategy: assign_only_product_as_production
Applying strategy: clean_integer_codes
Applying strategy: drop_unspecified_subcategories
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: update_ecoinvent_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: link_technosphere_by_activity_hash
Applied 11 strategies in 1.04 seconds


In [27]:
from bw2io.strategies.special import add_dummy_processes_and_rename_exchanges
ei.apply_strategy(add_dummy_processes_and_rename_exchanges)

Applying strategy: add_dummy_processes_and_rename_exchanges


In [28]:
ei.migrate("biosphere-2-3-names")
ei.migrate("biosphere-2-3-categories")
ei.migrate('default-units')

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


In [29]:
import functools
from bw2io.strategies import link_iterable_by_fields
f = functools.partial(link_iterable_by_fields,
    other=bw.Database(bw.config.biosphere),
    kind='biosphere'
)
ei.apply_strategy(f)

Applying strategy: link_iterable_by_fields


In [30]:
ei.match_database(None, ignore_categories=True)

Applying strategy: link_iterable_by_fields


Brute force approach - don't have time/energy for individual flows

In [16]:
ei.add_unlinked_flows_to_biosphere_database()

Writing activities to SQLite3 database:


Vacuuming database 


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/25/2019 12:44:15
  Finished: 07/25/2019 12:44:15
  Total time elapsed: 00:00:00
  CPU %: 82.40
  Memory %: 1.17
Applying strategy: link_iterable_by_fields


In [35]:
[x for x in ei.unlinked if x['name'] == 'Gasoline, combusted in equipement, at pulp and paper mill (EXCL.)']

[{'categories': (),
  'location': 'RNA',
  'unit': 'litre',
  'name': 'Gasoline, combusted in equipement, at pulp and paper mill (EXCL.)',
  'type': 'technosphere',
  'comment': 'Gasoline and kerosene, assumed to be gasoline.',
  'uncertainty type': 0,
  'amount': 0.19681,
  'loc': 0.19681}]

In [41]:
[x['name'] for x in ei.data if 'potato' in x['name'].lower()]

['Potato, whole plant, at field', 'potato starch, at plant']

In [32]:
ei.write_excel()

Wrote matching file to:
/Users/cmutel/Library/Application Support/Brightway3/US-EPA.ce86afc77ae88c7a59586427252c299c/output/db-matching-US-LCI.xlsx


In [31]:
ei.statistics()

1204 datasets
31262 exchanges
1151 unlinked exchanges
  Type production: 282 unique unlinked exchanges
  Type substitution: 1 unique unlinked exchanges
  Type technosphere: 211 unique unlinked exchanges


(1204, 31262, 1151)

There are a lot of misspellings, and a lot of links to simply missing activities. For now we ignore them, as we only really want the activity metadata for testing.

In [43]:
ei.drop_unlinked(True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.01 seconds


In [45]:
ei.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Title: Writing activities to SQLite3 database:
  Started: 07/25/2019 12:59:49
  Finished: 07/25/2019 12:59:51
  Total time elapsed: 00:00:02
  CPU %: 93.70
  Memory %: 1.39
Created database: US LCI


Brightway2 SQLiteBackend: US LCI